In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# FIX ME: Root dir as argument
root_dir = "../../data/picklerCheckpoints/"
is_compressed = True;

In [3]:
import numpy as np
import pandas as pd
import nbimporter
import geopandas as gpd
import commons.utility.pickle_util as pickle
from shapely.geometry import Point
import glob


Importing Jupyter notebook from D:\Academics\MLRepoNew\ML2020-Uber-Data-Analysis\commons\utility\pickle_util.ipynb


In [4]:
uberDataPath= '../data/uberData/'
weatherDataPath = "../data/weatherData/"
windDirectionPath = "../data/weatherData/"
windSpeedPath = "../data/weatherData/"
pressurePath = "../data/weatherData/"
humidityPath = "../data/weatherData/"
temperaturePath = "../data/weatherData/"
holidays = "../data/holidays/"
pickleStore = "../data/picklerCheckpoints/enrichedData/"

In [31]:
def generateHolidayLookupTable():
    picklerSave = "../data/picklerCheckpoints/holidays/holidays.p" 
    if (os.path.exists(picklerSave) or os.path.exists(picklerSave+'.pbz2')):
        frame = pickle.load(picklerSave, True)
        print("Holidays table loaded")
    else:
        print("Generating holidays lookup table")
        path = r'../data/holidays/' # use your path
        all_files = glob.glob(path + "/*.csv")

        li = []

        for filename in all_files:
            df = pd.read_csv(filename, index_col=None, header=0)
            li.append(df)

        frame = pd.concat(li, axis=0, ignore_index=True)
        frame = frame [['date-year', 'holiday']]
        frame = frame.drop_duplicates()
        frame['date-year'] = pd.to_datetime(frame['date-year'])
        frame['date-year'] = frame['date-year'].dt.strftime('%m/%d/%Y')
        pickle.save(frame, '../data/picklerCheckpoints/holidays/', 'holidays', True)
    return frame

In [21]:
def mergeAndSaveData(uberDataPaths, weatherDataPath, outputFileNames):
    # Get the holidays lookup table
    holidays = generateHolidayLookupTable()
    
    # Read the weather condition
    weatherData = pd.read_csv(weatherDataPath + 'weather_description.csv')
    weatherData["datetime"] = pd.to_datetime(weatherData['datetime'])
    weatherData['datetime'] = weatherData['datetime'].dt.strftime('%m-%d-%Y:%H')
    weatherData = weatherData[['datetime', 'New York']].drop_duplicates().dropna()
    weatherData.rename(columns = {'New York':'weatherCondition'}, inplace = True) 

    #Read the humidity readings
    humidityDf = pd.read_csv(weatherDataPath + 'humidity.csv')
    humidityDf["datetime"] = pd.to_datetime(humidityDf['datetime'])
    humidityDf['datetime'] = humidityDf['datetime'].dt.strftime('%m-%d-%Y:%H')
    humidityDf = humidityDf[['datetime', 'New York']].drop_duplicates().dropna()
    humidityDf.rename(columns = {'New York':'humidity'}, inplace = True) 

    #Read the pressure readings
    pressureDf = pd.read_csv(weatherDataPath + 'pressure.csv')
    pressureDf["datetime"] = pd.to_datetime(pressureDf['datetime'])
    pressureDf['datetime'] = pressureDf['datetime'].dt.strftime('%m-%d-%Y:%H')
    pressureDf = pressureDf[['datetime', 'New York']].drop_duplicates().dropna()
    pressureDf.rename(columns = {'New York':'pressure'}, inplace = True) 

    #Read the temperature readings
    temperatureDf = pd.read_csv(weatherDataPath + 'temperature.csv')
    temperatureDf["datetime"] = pd.to_datetime(temperatureDf['datetime'])
    temperatureDf['datetime'] = temperatureDf['datetime'].dt.strftime('%m-%d-%Y:%H')
    temperatureDf = temperatureDf[['datetime', 'New York']].drop_duplicates().dropna()
    temperatureDf.rename(columns = {'New York':'temperature'}, inplace = True) 

    #Read the wind direction readings
    windDirectionDf = pd.read_csv(weatherDataPath + 'wind_direction.csv')
    windDirectionDf["datetime"] = pd.to_datetime(windDirectionDf['datetime'])
    windDirectionDf['datetime'] = windDirectionDf['datetime'].dt.strftime('%m-%d-%Y:%H')
    windDirectionDf = windDirectionDf[['datetime', 'New York']].drop_duplicates().dropna()
    windDirectionDf.rename(columns = {'New York':'windDirection'}, inplace = True) 


    #Read the wind speed readings
    windSpeedDf = pd.read_csv(weatherDataPath + 'wind_speed.csv')
    windSpeedDf["datetime"] = pd.to_datetime(windSpeedDf['datetime'])
    windSpeedDf['datetime'] = windSpeedDf['datetime'].dt.strftime('%m-%d-%Y:%H')
    windSpeedDf = windSpeedDf[['datetime', 'New York']].drop_duplicates().dropna()
    windSpeedDf.rename(columns = {'New York':'windSpeed'}, inplace = True) 

    #loop through each uber input data file and introduce the new features to it.
    for i in range(0, len(uberDataPaths)):
        uberDataDF = pd.read_csv(uberDataPaths[i])
        uberDataDF['datetime'] = pd.to_datetime(uberDataDF['Date/Time'])
        uberDataDF['datetime'] = uberDataDF['datetime'].dt.strftime('%m-%d-%Y:%H')
        uberDataDF['date-year'] = pd.to_datetime(uberDataDF['Date/Time'])
        uberDataDF['date-year'] = uberDataDF['date-year'].dt.strftime('%m/%d/%Y')

        mergedDF = uberDataDF.merge(weatherData, on='datetime', how='left')
        mergedDF = mergedDF.merge(humidityDf, on='datetime', how='left')
        mergedDF = mergedDF.merge(pressureDf, on='datetime', how='left')
        mergedDF = mergedDF.merge(temperatureDf, on='datetime', how='left')
        mergedDF = mergedDF.merge(windDirectionDf, on='datetime', how='left')
        mergedDF = mergedDF.merge(windSpeedDf, on='datetime', how='left')
        mergedDF = mergedDF.merge(holidays, on='date-year', how='left').drop(['date-year'],axis = 1)

        pickle.save(mergedDF, '../data/picklerCheckpoints/enrichedData/', outputFileNames[i], True)
    print("Enrichment process has finished successfully")


In [7]:
uberDataPaths = [uberDataPath + 'uber-raw-data-apr14.csv', 
                 uberDataPath + 'uber-raw-data-may14.csv',
                 uberDataPath + 'uber-raw-data-jun14.csv',
                 uberDataPath + 'uber-raw-data-jul14.csv',
                 uberDataPath + 'uber-raw-data-aug14.csv',
                 uberDataPath + 'uber-raw-data-sep14.csv']

outputFileNames = ['uber-enriched-data-apr14', 
                   'uber-enriched-data-may14',
                   'uber-enriched-data-jun14',
                   'uber-enriched-data-jul14',
                   'uber-enriched-data-aug14',
                   'uber-enriched-data-sep14']

#mergeAndSaveData(uberDataPaths, weatherDataPath, outputFileNames)

In [8]:
def getUberDataforMonth(month):
    uberDataPath= '../data/uberData/'
    pickleStore = "../data/picklerCheckpoints/enrichedData/"
    weatherDataPath = "../data/weatherData/"
    validMonths = ['apr', 'may', 'jun', 'jul','aug', 'sep']
    data = None
    path = pickleStore + 'uber-enriched-data-'+ month +'14.p'
    if(month in validMonths):
        if (os.path.exists(path) or os.path.exists(path+'.pbz2')):
            data = pickle.load(path, True)
            print("Dataset loaded")
        else:
            print("Missing data--Regenerating the eneriched data")
            mergeAndSaveData([uberDataPath + 'uber-raw-data-'+ month +'14.csv'], weatherDataPath, ['uber-enriched-data-'+ month +'14'])
            data = pickle.load(path, True)
            print("Dataset loaded")
    else: 
        print("Invalid month entered \n Valid months are: ")
        print(*validMonths, sep='\n')
    return data

In [32]:
# df = getUberDataforMonth('apr')

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Loading file ../data/picklerCheckpoints/enrichedData/uber-enriched-data-apr14.p
Decompressing ../data/picklerCheckpoints/enrichedData/uber-enriched-data-apr14.p.pbz2..............Done
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Dataset loaded


In [33]:
# holidays = generateHolidayLookupTable()

Generating holidays lookup table
Result Saved in ../data/picklerCheckpoints/holidays/holidays.p


In [34]:
# holidays

,date-year,holiday
0,04/01/2014,weekday
1011,04/02/2014,weekday
2347,04/03/2014,weekday
3829,04/04/2014,weekday
5656,04/05/2014,weekend
...,...,...
3535996,09/26/2014,weekday
3536949,09/27/2014,weekend
3537954,09/28/2014,weekend
3538688,09/29/2014,weekday


In [35]:
# df['date-year'] = pd.to_datetime(df['Date/Time'])
# df['date-year'] = df['date-year'].dt.strftime('%m/%d/%Y')

In [36]:
# df = df.merge(holidays, on='date-year', how='left')

In [37]:
# df

,Date/Time,Lat,Lon,Base,datetime,weatherCondition,humidity,pressure,temperature,windDirection,windSpeed,holiday_x,date-year,holiday_y
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512,04-01-2014:00,scattered clouds,40.0,1016.0,282.960,330.0,5.0,NaN,04/01/2014,weekday
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512,04-01-2014:00,scattered clouds,40.0,1016.0,282.960,330.0,5.0,NaN,04/01/2014,weekday
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512,04-01-2014:00,scattered clouds,40.0,1016.0,282.960,330.0,5.0,NaN,04/01/2014,weekday
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512,04-01-2014:00,scattered clouds,40.0,1016.0,282.960,330.0,5.0,NaN,04/01/2014,weekday
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512,04-01-2014:00,scattered clouds,40.0,1016.0,282.960,330.0,5.0,NaN,04/01/2014,weekday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564511,4/30/2014 23:22:00,40.7640,-73.9744,B02764,04-30-2014:23,very heavy rain,95.0,1033.0,284.317,118.0,5.0,NaN,04/30/2014,weekday
564512,4/30/2014 23:26:00,40.7629,-73.9672,B02764,04-30-2014:23,very heavy rain,95.0,1033.0,284.317,118.0,5.0,NaN,04/30/2014,weekday
564513,4/30/2014 23:31:00,40.7443,-73.9889,B02764,04-30-2014:23,very heavy rain,95.0,1033.0,284.317,118.0,5.0,NaN,04/30/2014,weekday
564514,4/30/2014 23:32:00,40.6756,-73.9405,B02764,04-30-2014:23,very heavy rain,95.0,1033.0,284.317,118.0,5.0,NaN,04/30/2014,weekday


In [9]:
# df.describe()
# data = df.copy()

In [10]:
import bokeh, bokeh.plotting, bokeh.models
from bokeh.io import output_notebook, show
output_notebook()
import sklearn.preprocessing
from shapely.geometry import Point
import rtree

coord_system = {'init': 'epsg:4326'}

df = gpd.read_file('../data/NewYorkCoords/taxi_zones.shp').to_crs(coord_system)
df = df.drop(['Shape_Area', 'Shape_Leng', 'OBJECTID'], axis=1)
pd.set_option('display.max_colwidth', 50)

df

Loading BokehJS ...

D:\Anaconda\envs\tensorflow\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,zone,LocationID,borough,geometry
0,Newark Airport,1,EWR,"POLYGON ((-74.18445 40.69500, -74.18449 40.695..."
1,Jamaica Bay,2,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((-73.84793 40.87134, -73.84725 40.870..."
3,Alphabet City,4,Manhattan,"POLYGON ((-73.97177 40.72582, -73.97179 40.725..."
4,Arden Heights,5,Staten Island,"POLYGON ((-74.17422 40.56257, -74.17349 40.562..."
...,...,...,...,...
258,Woodlawn/Wakefield,259,Bronx,"POLYGON ((-73.85107 40.91037, -73.85207 40.909..."
259,Woodside,260,Queens,"POLYGON ((-73.90175 40.76077, -73.90147 40.759..."
260,World Trade Center,261,Manhattan,"POLYGON ((-74.01333 40.70503, -74.01327 40.704..."
261,Yorkville East,262,Manhattan,"MULTIPOLYGON (((-73.94383 40.78286, -73.94376 ..."


In [11]:
df2 = df.copy()
df2['geometry'] = df.geometry.envelope
df2['borough_categ'] = sklearn.preprocessing.LabelEncoder().fit_transform(df2['borough'])
gjds2 = bokeh.models.GeoJSONDataSource(geojson=df2.to_json())

TOOLS = "pan,wheel_zoom,reset,hover,save"

p = bokeh.plotting.figure(title="NYC Taxi Districts Bounding Boxes", tools=TOOLS,
    x_axis_location=None, y_axis_location=None)

color_mapper = bokeh.models.LinearColorMapper(palette=bokeh.palettes.Viridis256)

p.patches('xs', 'ys', 
          fill_color={'field': 'borough_categ', 'transform': color_mapper},
          fill_alpha=0.7, line_color="black", line_width=0.5,          
          source=gjds2)

p.grid.grid_line_color = None

hover = p.select_one(bokeh.models.HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = u"""
<div> 
    <div class="bokeh_hover_tooltip">Name : @zone</div>
    <div class="bokeh_hover_tooltip">Borough : @borough</div>
    <div class="bokeh_hover_tooltip">Zone ID : @LocationID</div>
    <div class="bokeh_hover_tooltip">(Lon, Lat) : ($x ˚E, $y ˚N)</div>
</div>
"""

p.circle([-73.966,], [40.78,], size=10, fill_color='magenta', line_color='yellow', line_width=1, alpha=1.0)

show(p)

In [12]:
# gdf = gpd.GeoDataFrame(
#     data, geometry=gpd.points_from_xy(data.Lon, data.Lat), crs={'init': 'epsg:4326'})

In [13]:
# mappedData = gpd.sjoin(gdf, 
#           df2,
#           how='left', op='within')

In [ ]:
# mappedData

In [23]:
generateHolidayLookupTable()

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Loading file ../data/picklerCheckpoints/holidays/holidays.p
Decompressing ../data/picklerCheckpoints/holidays/holidays.p.pbz2..............Done
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Holidays table loaded


,date-year,holiday
0,4/1/2014,weekday
1011,4/2/2014,weekday
2347,4/3/2014,weekday
3829,4/4/2014,weekday
5656,4/5/2014,weekend
...,...,...
3535996,9/26/2014,weekday
3536949,9/27/2014,weekend
3537954,9/28/2014,weekend
3538688,9/29/2014,weekday
